In [1]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import nltk
from nltk.corpus import stopwords
STOP_WORDS = set(stopwords.words('german'))
new_stopwords = ['|lbr|', 'ja']
STOP_WORDS = STOP_WORDS.union(new_stopwords)

/home/mackenzie/anaconda3/lib/python3.7/site-packages/pyparsing.py:2725: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/mackenzie/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [3]:
h2o.init()
#nltk.download('stopwords')  # might need if running nltk + stopwords for the first time
train_data = h2o.upload_file("/home/mackenzie/Downloads/GermanCleanedTrainingData.csv") # check if path correct
test_data = h2o.upload_file("/home/mackenzie/Downloads/GermanCleanedTestingData.csv") # check if path correct

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,7 days 19 hours 38 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_mackenzie_dm3nzl
H2O cluster total nodes:,1
H2O cluster free memory:,1021 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
print("Break train tweets into sequence of words")
train_words = tokenize(train_data["tweet"])

print("Break test tweets into sequence of words")
test_words = tokenize(test_data["tweet"])

print("Build word2vec model for train")
w2v_model_one = H2OWord2vecEstimator(sent_sample_rate=0.0, epochs=10)
w2v_model_one.train(training_frame=train_words)

print("Build word2vec model for test")
w2v_model_two = H2OWord2vecEstimator(sent_sample_rate=0.0, epochs=10)
w2v_model_two.train(training_frame=test_words)

print("Calculate a vector for each train tweet")
tweet_vecs_train = w2v_model_one.transform(train_words, aggregate_method="AVERAGE")

print("Calculate a vector for each test tweet")
tweet_vecs_test = w2v_model_two.transform(test_words, aggregate_method="AVERAGE")



Break train tweets into sequence of words
Break test tweets into sequence of words
Build word2vec model for train
word2vec Model Build progress: |██████████████████████████████████████████| 100%
Build word2vec model for test
word2vec Model Build progress: |██████████████████████████████████████████| 100%
Calculate a vector for each train tweet
Calculate a vector for each test tweet


In [5]:
train_tweets_pd = h2o.as_list(tweet_vecs_train, use_pandas=True)
train_labels_pd = h2o.as_list(train_data["labels"], use_pandas=True)
test_tweets_pd = h2o.as_list(tweet_vecs_test, use_pandas=True)
test_labels_pd = h2o.as_list(test_data["labels"], use_pandas=True)

#print(type(train_tweets_pd))
X_train = train_tweets_pd
y_train = train_labels_pd
X_test = test_tweets_pd
y_test = test_labels_pd


<class 'pandas.core.frame.DataFrame'>


In [6]:
print("start automl!")
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train, dataset_name='german_data') # feat_type=classification is another param we might use?
y_hat = automl.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))
automl.cv_results_
automl.sprint_statistics()
automl.show_models()

start automl!


/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/automl.py:887: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-07-04 10:51:35,949:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-04 10:51:35,957:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-04 10:51:37,966:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-04 10:51:39,971:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!


/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-07-04 10:58:45,961:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-07-04 10:58:45,961:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/

/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/

/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


1
['/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000005.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000006.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000007.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000008.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000009.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000010.ensemble', '/tmp/autosklearn_tmp_12607_8734/.auto-sklearn/ensembles/1.0000000011.ensemble', '/tmp/autosklearn_tmp_126

"[(0.380000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'adaboost', 'imputation:strategy': 'mean', 'preprocessor:__choice__': 'select_rates', 'rescaling:__choice__': 'normalize', 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'False', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_rate': 0.6364087198858188, 'classifier:adaboost:max_depth': 4, 'classifier:adaboost:n_estimators': 284, 'preprocessor:select_rates:alpha': 0.4086188588455056, 'preprocessor:select_rates:mode': 'fwe', 'preprocessor:select_rates:score_func': 'f_classif'},\ndataset_properties={\n  'task': 2,\n  'sparse': False,\n  'multilabel': False,\n  'multiclass': True,\n  'target_type': 'classification',\n  'signed': False})),\n(0.380000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'adaboost', '